In [34]:
#basic libraries import
import pandas as pd
import numpy as np

In [35]:
data=pd.read_csv("test_data.csv",parse_dates=['Timestamp'])

In [36]:
data.reset_index(level=0, inplace=True) 

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   index            20 non-null     int64         
 1   Timestamp        20 non-null     datetime64[ns]
 2   Ignition_Status  20 non-null     int64         
 3   Engine_speed     20 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 768.0 bytes


In [38]:
data

,index,Timestamp,Ignition_Status,Engine_speed
0,0,2020-06-16 10:45:00,0,0
1,1,2020-06-16 10:45:00,0,0
2,2,2020-06-16 10:45:00,0,0
3,3,2020-06-16 10:45:00,0,0
4,4,2020-06-16 10:46:00,1,0
5,5,2020-06-16 10:46:00,1,0
6,6,2020-06-16 10:46:00,1,0
7,7,2020-06-16 10:46:00,1,0
8,8,2020-06-16 10:46:00,0,0
9,9,2020-06-16 10:46:00,0,0


In [39]:
data['Ignition_Status'].value_counts()

0    11
1     9
Name: Ignition_Status, dtype: int64

In [40]:
data.iloc[:]

,index,Timestamp,Ignition_Status,Engine_speed
0,0,2020-06-16 10:45:00,0,0
1,1,2020-06-16 10:45:00,0,0
2,2,2020-06-16 10:45:00,0,0
3,3,2020-06-16 10:45:00,0,0
4,4,2020-06-16 10:46:00,1,0
5,5,2020-06-16 10:46:00,1,0
6,6,2020-06-16 10:46:00,1,0
7,7,2020-06-16 10:46:00,1,0
8,8,2020-06-16 10:46:00,0,0
9,9,2020-06-16 10:46:00,0,0


In [41]:
from IPython.core.debugger import set_trace

In [52]:
trip_id=1000
Trip_info=[]
start_index=0
stop_index=0
rows=0
x=0
for rows in data.index:
    for rows in data.index[start_index:]:

        #set_trace()
        if(data['Ignition_Status'].iloc[rows]== 1):
            start_index=data['index'].iloc[rows]
            for x in data.index[rows:]:
                if(data['Ignition_Status'].iloc[x]==0):
                    stop_index = data['index'].iloc[x]
                    break
            Trip_info_temp=[trip_id,start_index,stop_index]
            Trip_info.append(Trip_info_temp)
            start_index=stop_index
            trip_id +=1
            break
        continue

        
print(Trip_info)


[[1000, 4, 8], [1001, 11, 16]]
